In [15]:
!pip install gensim
!pip install spacy
!pip install contractions

In [16]:
# Data Handling & Plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

# NLTK for preprocessing
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

# Scikit-learn for Vectorization, Classification, Clustering, and Evaluation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
from sklearn.metrics import ConfusionMatrixDisplay

# Gensim for Topic Modeling
import gensim
from gensim import corpora
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

# spaCy for NER
import spacy
from spacy import displacy

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import pandas as pd

# df = pd.read_csv('youtoxic_english_1000.csv') ใช้อันนี้ถ้าทำในvs code
df = pd.read_csv('/content/drive/MyDrive/youtoxic_english_1000.csv')
print("DataFrame created:")
display(df.head())

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [19]:
print(df['Text'].iloc[3])

There are a very large number of people who do not like police officers. They are called Criminals and its the reason we have police officers. The fact that Criminals do not like police officers is a testament to the good work that police officers do in protecting the public. When our children or our family are in danger, we do not hessitate to call for help, and we call the Police. Its about time people stopped complaining and started to give the Police some respect for the hard work and dedication that often requires them to put their lives on the line, to serve the public.


In [20]:
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
stop_words.remove('not')
import contractions

def normalize_apostrophe(text):
    text = text.replace("\n", " ")
    text = text.replace("’", "'")
    text = text.replace("`", "'")
    text = text.replace("‘", "'")
    text = re.sub(r"\b[Dd]ont\b", "don't", text)
    return text

def get_wordnet_pos_hw(treebank_tag):
    if treebank_tag.startswith('J'): return wordnet.ADJ
    elif treebank_tag.startswith('V'): return wordnet.VERB
    elif treebank_tag.startswith('N'): return wordnet.NOUN
    elif treebank_tag.startswith('R'): return wordnet.ADV
    else: return wordnet.NOUN

def preprocess_text_hw(text):
    # text = contractions.fix(text)
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    clean_tokens = []
    for word, tag in pos_tags:
        word = re.sub(r'[^a-zA-Z]', '', word).lower()
        if word and word not in stop_words and len(word) > 2:
            word = lemmatizer.lemmatize(word, pos=get_wordnet_pos_hw(tag))
            clean_tokens.append(word)
    return ' '.join(clean_tokens)


df['Text'] = df['Text'].apply(normalize_apostrophe)
df['Text'] = df['Text'].astype(str).apply(contractions.fix)
df['Text'] = df['Text'].apply(preprocess_text_hw)
display(df.head())
print(df['Text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,people would take step back not make case not ...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,law enforcement not train shoot apprehend trai...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,not reckon black live matter banner hold white...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,large number people not like police officer ca...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,arab dude absolutely right not shot extra time...,False,False,False,False,False,False,False,False,False,False,False,False


0      people would take step back not make case not ...
1      law enforcement not train shoot apprehend trai...
2      not reckon black live matter banner hold white...
3      large number people not like police officer ca...
4      arab dude absolutely right not shot extra time...
                             ...                        
995                       remember send national defence
996    stats not represent problem race baiting attit...
997                   quote mother wow hit hard accurate
998                                         video racist
999                              god narrator annoy lisp
Name: Text, Length: 1000, dtype: object


In [21]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

In [27]:
# import gensim.downloader as api
# fasttext = api.load("fasttext-wiki-news-subwords-300") อันนี้ word embedding

In [28]:
# def sentence_embedding(sentence):
#     words = sentence.split()อันนี้ของ word embedding
#     vectors = []

#     for word in words:
#         vectors.append(fasttext[word])

#     if len(vectors) == 0:
#         return np.zeros(fasttext.vector_size)

#     return np.mean(vectors, axis=0)

In [29]:
# X_embedded = np.array([sentence_embedding(text) for text in df["Text"]])

In [26]:
print(df['Text'].iloc[3])

large number people not like police officer call criminal reason police officer fact criminal not like police officer testament good work police officer protect public child family danger not hessitate call help call police time people stop complain start give police respect hard work dedication often require put life line serve public


In [23]:
vectorizer = TfidfVectorizer(
    min_df=5,
    max_df=0.7,
    ngram_range=(1, 1)
)

tfidf_matrix = vectorizer.fit_transform(df['Text'])
feature_names = vectorizer.get_feature_names_out()

print(f"Created a TF-IDF matrix of shape: {tfidf_matrix.shape}")
print(f"(Documents, Features/Vocabulary)")
print(f"Vocabulary size: {len(feature_names)}")
print(f"Sample features: {feature_names[500:510]}")

Created a TF-IDF matrix of shape: (1000, 632)
(Documents, Features/Vocabulary)
Vocabulary size: 632
Sample features: ['show' 'sick' 'side' 'sign' 'simple' 'simply' 'since' 'single' 'sit'
 'situation']


In [24]:
label_cols = [
    "IsToxic",
    "IsAbusive",
    "IsThreat",
    "IsProvocative",
    "IsObscene",
    "IsHatespeech",
    "IsRacist",
    "IsNationalist",
    "IsSexist",
    "IsHomophobic",
    "IsReligiousHate",
    "IsRadicalism"
]
print(df.columns)

Index(['CommentId', 'VideoId', 'Text', 'IsToxic', 'IsAbusive', 'IsThreat',
       'IsProvocative', 'IsObscene', 'IsHatespeech', 'IsRacist',
       'IsNationalist', 'IsSexist', 'IsHomophobic', 'IsReligiousHate',
       'IsRadicalism'],
      dtype='object')


In [25]:
X = tfidf_matrix
y = df[label_cols]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.4,
    random_state=42,
)

X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test,
    test_size=0.5,
    random_state=42,
)

print(f"X shape: {X.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_train shape: {X_val.shape}")

X shape: (1000, 632)
X_train shape: (600, 632)
X_test shape: (200, 632)
X_train shape: (200, 632)
